In [66]:
import os
import numpy as np
from scipy.linalg import eigh
from scipy import sparse
from node_embedding_attack.utils import *
from node_embedding_attack.embedding import *
from node_embedding_attack.perturbation_attack import *

### Load and preprocess the data

In [67]:
data_dir = os.path.expanduser("~/data/cora/")
cora_location = os.path.expanduser(os.path.join(data_dir, "cora.cites"))
g_nx = nx.read_edgelist(path=cora_location)
adj_matrix = nx.to_numpy_array(g_nx)
adj_matrix = sparse.csr_matrix(adj_matrix)

In [68]:
adj_matrix.shape

(2708, 2708)

In [69]:
type(adj_matrix)

scipy.sparse.csr.csr_matrix

In [18]:
if False:
    graph = load_dataset('data/cora.npz')
    adj_matrix = graph['adj_matrix']
    labels = graph['labels']

    adj_matrix, labels = standardize(adj_matrix, labels)
    n_nodes = adj_matrix.shape[0]

### Set hyperparameters

In [3]:
n_flips = 1000
dim = 32
window_size = 5

### Generate candidate edge flips

In [4]:
candidates = generate_candidates_removal(adj_matrix=adj_matrix)

### Compute simple baselines

In [5]:
b_eig_flips = baseline_eigencentrality_top_flips(adj_matrix, candidates, n_flips)
b_deg_flips = baseline_degree_top_flips(adj_matrix, candidates, n_flips, True)
b_rnd_flips = baseline_random_top_flips(candidates, n_flips, 0)

### Compute adversarial flips using eigenvalue perturbation

In [6]:
our_flips = perturbation_top_flips(adj_matrix, candidates, n_flips, dim, window_size)

In [7]:
our_flips

array([[ 326, 1933],
       [ 550,  569],
       [ 705,  910],
       ...,
       [ 735, 2802],
       [ 840, 1968],
       [ 254, 2259]], dtype=int32)

### Evaluate classification performance using the skipgram objective

In [7]:
for flips, name in zip([None, b_rnd_flips, b_deg_flips, None, our_flips],
                             ['cln', 'rnd', 'deg', 'eig', 'our']):
    
    if flips is not None:
        adj_matrix_flipped = flip_candidates(adj_matrix, flips)
    else:
        adj_matrix_flipped = adj_matrix
        
    embedding = deepwalk_skipgram(adj_matrix_flipped, dim, window_size=window_size)
    f1_scores_mean, _ = evaluate_embedding_node_classification(embedding, labels)
    print('{}, F1: {:.2f} {:.2f}'.format(name, f1_scores_mean[0], f1_scores_mean[1]))

cln, F1: 0.81 0.77


ValueError: shape mismatch in assignment

### Evaluate classification performance using the SVD objective

In [8]:
for flips, name in zip([None, b_rnd_flips, b_deg_flips, None, our_flips],
                             ['cln', 'rnd', 'deg', 'eig', 'our']):
    
    if flips is not None:
        adj_matrix_flipped = flip_candidates(adj_matrix, flips)
    else:
        adj_matrix_flipped = adj_matrix
        
    embedding, _, _, _ = deepwalk_svd(adj_matrix_flipped, window_size, dim)
    f1_scores_mean, _ = evaluate_embedding_node_classification(embedding, labels)
    print('{}, F1: {:.2f} {:.2f}'.format(name, f1_scores_mean[0], f1_scores_mean[1]))

ValueError: shape mismatch in assignment

### Store attacked graph

In [70]:
def attack_graph(adj_matrix, n_flips, dim, window_size, seed=0, method="add"):
    if method=="add":
        candidates = generate_candidates_addition(adj_matrix=adj_matrix,
                                                  n_candidates=n_flips, 
                                                  seed=seed)
    else:
        candidates = generate_candidates_removal(adj_matrix=adj_matrix, 
                                                 seed=seed)
    our_flips = perturbation_top_flips(adj_matrix, candidates, n_flips, dim, window_size)
    #
    A = np.array(adj_matrix.todense())
    
    A_flipped = A.copy()
    A_flipped[candidates[:, 0], candidates[:, 1]] = 1 - A[candidates[:, 0], candidates[:, 1]]
    A_flipped[candidates[:, 1], candidates[:, 0]] = 1 - A[candidates[:, 1], candidates[:, 0]]
    
    return A_flipped

In [71]:
n_flips = 1000
dim = 32
window_size = 5

In [72]:
candidates = generate_candidates_removal(adj_matrix=adj_matrix)

In [73]:
our_flips = perturbation_top_flips(adj_matrix, candidates, n_flips, dim, window_size)

In [74]:
data = 'cora'
ele = 'attack'
#corrupted_A = corrupt_adjacency(A, ele, l)
dir_name = os.path.join("attacked_datasets",data,ele)
print(dir_name)
i = 1

attacked_datasets/cora/attack


In [75]:
print(type(adj_matrix))
A = np.array(adj_matrix.todense())
print(type(A))

<class 'scipy.sparse.csr.csr_matrix'>
<class 'numpy.ndarray'>


In [76]:
# This flips the selected edges
#adj_matrix_flipped = flip_candidates(A, our_flips)
A_flipped = A.copy()
A_flipped[candidates[:, 0], candidates[:, 1]] = 1 - A[candidates[:, 0], candidates[:, 1]]
A_flipped[candidates[:, 1], candidates[:, 0]] = 1 - A[candidates[:, 1], candidates[:, 0]]

In [77]:
if not os.path.exists(dir_name):
    os.makedirs(dir_name)
file_name = data + "_" + ele + "_"+str(n_flips)+"_v"+str(i)
print(f"file_name: {file_name}")
np.save(os.path.join(dir_name,file_name), A_flipped)

file_name: cora_attack_1000_v1


In [78]:
num_flips = [ -2000, -1000, -500, 500, 1000, 2000, 5000 ]

In [79]:
for n_flips in num_flips:
    print(f"Calculating for n_flips={n_flips}")
    if n_flips < 0:
        method = "remove"
        n_flips = -n_flips
    else:
        method = "add"
    A_flipped = attack_graph(adj_matrix=adj_matrix, 
                             n_flips=n_flips, 
                             dim=dim, 
                             window_size=window_size, 
                             method=method, 
                             seed=0)
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    file_name = data + "_" + ele + "_"+str(n_flips)+"_"+method #+"_v"+str(i)
    print(f"file_name: {file_name}")
    np.save(os.path.join(dir_name,file_name), A_flipped)
    
    graph = nx.from_numpy_array(A_flipped)
    file_name += ".gpickle"
    nx.write_gpickle(graph, os.path.join(dir_name, file_name))

Calculating for n_flips=-2000
file_name: cora_attack_2000_remove
Calculating for n_flips=-1000
file_name: cora_attack_1000_remove
Calculating for n_flips=-500
file_name: cora_attack_500_remove
Calculating for n_flips=500
file_name: cora_attack_500_add
Calculating for n_flips=1000
file_name: cora_attack_1000_add
Calculating for n_flips=2000
file_name: cora_attack_2000_add
Calculating for n_flips=5000
file_name: cora_attack_5000_add


In [80]:
adj_matrix.shape

(2708, 2708)